In [17]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pickle
from sklearn.metrics import classification_report
from sklearn.datasets import make_circles, make_moons
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
with open('cifar/cifar-100-python/train', 'rb') as f:
    data_train = pickle.load(f, encoding='latin1')
with open('cifar/cifar-100-python/test', 'rb') as f:
    data_test = pickle.load(f, encoding='latin1')

# Здесь указать ваши классы по варианту!!!
# 1. Номер группы + 15
# 2. Номер варианта + 56
# 3. ИУ5 (Номер варианта + 21); ГУИМЦ (80); Иностранцы (90)
# (39 — клавиатуры, 58 — небо, 23 — внедорожники)
CLASSES = [39, 58, 23]

# массив
train_X = data_train['data'].reshape(-1, 3, 32, 32)
train_X = np.transpose(train_X, [0, 2, 3, 1]) # NCHW -> NHWC
train_y = np.array(data_train['fine_labels'])
mask = np.isin(train_y, CLASSES) # фильтруем данные, выбираем только те, которые относятся к нашим классам

train_X = train_X[mask].copy()
train_y = train_y[mask].copy()
train_y = np.unique(train_y, return_inverse=1)[1]
del data_train

test_X = data_test['data'].reshape(-1, 3, 32, 32)
test_X = np.transpose(test_X, [0, 2, 3, 1])
test_y = np.array(data_test['fine_labels'])
mask = np.isin(test_y, CLASSES)

test_X = test_X[mask].copy()
test_y = test_y[mask].copy()
test_y = np.unique(test_y, return_inverse=1)[1]
del data_test

In [19]:
class Normalize(nn.Module):
    
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.tensor(mean)
        self.std = torch.tensor(std)

    def forward(self, input):
        x = input / 255.0
        x = x - self.mean
        x = x / self.std
        return torch.flatten(x, start_dim=1) # nhwc -> nm

class Cifar100_MLP(nn.Module):
    def __init__(self, _seq : nn.Sequential):
        super(Cifar100_MLP, self).__init__()
        # https://blog.jovian.ai/image-classification-of-cifar100-dataset-using-pytorch-8b7145242df1
        self.norm = Normalize([0.5074,0.4867,0.4411],[0.2011,0.1987,0.2025])
        self.seq = _seq
        #self.seq = nn.Sequential(
        #    nn.Linear(32*32*3, hidden_size),
        #    nn.ReLU(),
        #    nn.Linear(hidden_size, classes),
        #)

    def forward(self, input):
        x = self.norm(input)
        return self.seq(x)

In [58]:
def task_3(_batch_size, _epochs, _classes: int, _learning_rate:float, seq: nn.Sequential):
  batch_size = _batch_size
  dataloader = {}

  for (X, y), part in zip([(train_X, train_y), (test_X, test_y)], ['train', 'test']):
      tensor_x = torch.Tensor(X)
      tensor_y = F.one_hot(torch.Tensor(y).to(torch.int64), num_classes=_classes)/1.
      dataset = TensorDataset(tensor_x, tensor_y)
      dataloader[part] = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  model = Cifar100_MLP(seq)

  print(model)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=_learning_rate)

  EPOCHS = _epochs
  # train data set
  steps_per_epoch_train = len(dataloader['train'])
  # test data set (валидация)
  steps_per_epoch_test = len(dataloader['test'])

  losses_train_x = []
  losses_train_y = []
  losses_test_x = []
  losses_test_y = []

  for epoch in range(EPOCHS):  # проход по набору данных несколько раз
      running_loss = 0.0
      model.train()

      for i, batch in enumerate(dataloader['train'], 0):
          # получение одного минибатча; batch это двуэлементный список из [inputs, labels]
          inputs, labels = batch

          # очищение прошлых градиентов с прошлой итерации
          optimizer.zero_grad()

          # прямой + обратный проходы + оптимизация
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          
          loss.backward()
          optimizer.step()

          # для подсчёта статистик
          running_loss += loss.item()

      # график
      losses_train_x.append(epoch)
      losses_train_y.append(running_loss / steps_per_epoch_train)

      running_loss = 0.0
      model.eval() # модель переводится в evaluation mode (режим оценки)
      with torch.no_grad(): # отключение автоматического дифференцирования
          for i, batch in enumerate(dataloader['test'], 0):
              inputs, labels = batch

              outputs = model(inputs)
              loss = criterion(outputs, labels)
              running_loss += loss.item()

      # график
      losses_test_x.append(epoch)
      losses_test_y.append(running_loss / steps_per_epoch_test)

  print('Обучение закончено')
  
  min_losses_test_y = max(losses_test_y)
  min_losses_test_x = 0
  for i in range(0, len(losses_test_y)):
      if losses_test_y[i] < min_losses_test_y:
          min_losses_test_y = losses_test_y[i]
          min_losses_test_x = i
  
  print("Переобучение произошло на эпохе: ", min_losses_test_x)
  print("Ошибка в момент переобучения: ", min_losses_test_y)

  for part in ['train', 'test']:
    y_pred = []
    y_true = []
    with torch.no_grad(): # отключение автоматического дифференцирования
        for i, data in enumerate(dataloader[part], 0):
            inputs, labels = data

            outputs = model(inputs).detach().numpy()
            y_pred.append(outputs)
            y_true.append(labels.numpy())
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        print(part)
        print(classification_report(y_true.argmax(axis=-1), y_pred.argmax(axis=-1), digits=4, target_names=list(map(str, CLASSES))))
        print('-'*50)

  return model

In [34]:
BATCH_SIZE = 128
HIDDEN_SIZE = 32
LEARNING_RATE = 0.005
NUM_OF_CLASSES = 3
EPOCHS = 250
def default_sequention():
    return nn.Sequential(
            # полносвязный слой (входной - скрытый)
            nn.Linear(32*32*3, HIDDEN_SIZE),
            # функция активации для нейронов скрытого слоя
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(HIDDEN_SIZE, NUM_OF_CLASSES),
        )

In [36]:
task_3(BATCH_SIZE, EPOCHS, NUM_OF_CLASSES, LEARNING_RATE, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  91
Ошибка в моменте переобучения:  0.5351938704649607
train
              precision    recall  f1-score   support

          39     0.9920    0.9900    0.9910       500
          58     0.9920    0.9920    0.9920       500
          23     0.9980    1.0000    0.9990       500

    accuracy                         0.9940      1500
   macro avg     0.9940    0.9940    0.9940      1500
weighted avg     0.9940    0.9940    0.9940      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6967    0.8500    0.7658       100
          58     0.7442    0.6400    0.6882       100
          23     0.8696    0.8000    0.8333       100

    accuracy                    

In [37]:
task_3(BATCH_SIZE, 75, NUM_OF_CLASSES, LEARNING_RATE, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  72
Ошибка в моменте переобучения:  0.5693243046601614
train
              precision    recall  f1-score   support

          39     0.9010    0.9280    0.9143       500
          58     0.9367    0.8880    0.9117       500
          23     0.9511    0.9720    0.9614       500

    accuracy                         0.9293      1500
   macro avg     0.9296    0.9293    0.9291      1500
weighted avg     0.9296    0.9293    0.9291      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6783    0.7800    0.7256       100
          58     0.6905    0.5800    0.6304       100
          23     0.8416    0.8500    0.8458       100

    accuracy                    

In [38]:
# Уменьшили learning rate для повышения точности. Соответственно увеличили количество эпох
task_3(BATCH_SIZE, 375, NUM_OF_CLASSES, 0.001, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  314
Ошибка в моменте переобучения:  0.5744607051213583
train
              precision    recall  f1-score   support

          39     0.9202    0.9220    0.9211       500
          58     0.9249    0.9120    0.9184       500
          23     0.9526    0.9640    0.9583       500

    accuracy                         0.9327      1500
   macro avg     0.9326    0.9327    0.9326      1500
weighted avg     0.9326    0.9327    0.9326      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6639    0.8100    0.7297       100
          58     0.6923    0.5400    0.6067       100
          23     0.8200    0.8200    0.8200       100

    accuracy                   

In [59]:
# Обнаружено переобучение для прошлой итерации. Снизили количество эпох до 314.
result = task_3(BATCH_SIZE, 314, NUM_OF_CLASSES, 0.001, default_sequention())
# ВТОРОЙ СПОСОБ: сохранение всей архитектуры
PATH2 = 'cifar_cnn.pt'
torch.save(result, PATH2)
# загрузка
new_model_2 = torch.load(PATH2)
new_model_2.eval()

# входной тензор для модели
x = torch.randn(1, 32, 32, 3, requires_grad=True).to("cpu")
torch_out = result(x)

# экспорт модели
torch.onnx.export(result,               # модель
                  x,                   # входной тензор (или кортеж нескольких тензоров)
                  "cifar100_CNN_lr_1.onnx", # куда сохранить (либо путь к файлу либо fileObject)
                  export_params=True,  # сохраняет веса обученных параметров внутри файла модели
                  opset_version=9,     # версия ONNX
                  do_constant_folding=True,  # следует ли выполнять укорачивание констант для оптимизации
                  input_names = ['input'],   # имя входного слоя
                  output_names = ['output'],  # имя выходного слоя
                  dynamic_axes={'input' : {0 : 'batch_size'},    # динамичные оси, в данном случае только размер пакета
                                'output' : {0 : 'batch_size'}})

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  262
Ошибка в момент переобучения:  0.5599540869394938
train
              precision    recall  f1-score   support

          39     0.8706    0.8880    0.8792       500
          58     0.8921    0.8600    0.8758       500
          23     0.9429    0.9580    0.9504       500

    accuracy                         0.9020      1500
   macro avg     0.9019    0.9020    0.9018      1500
weighted avg     0.9019    0.9020    0.9018      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.7043    0.8100    0.7535       100
          58     0.7000    0.6300    0.6632       100
          23     0.8632    0.8200    0.8410       100

    accuracy                    

In [40]:
# Увеличение batch size в 1.5 раз 
task_3(192, 471, NUM_OF_CLASSES, 0.001, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  417
Ошибка в моменте переобучения:  0.5591932982206345
train
              precision    recall  f1-score   support

          39     0.9162    0.8960    0.9060       500
          58     0.9016    0.8980    0.8998       500
          23     0.9337    0.9580    0.9457       500

    accuracy                         0.9173      1500
   macro avg     0.9172    0.9173    0.9172      1500
weighted avg     0.9172    0.9173    0.9172      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.7321    0.8200    0.7736       100
          58     0.7204    0.6700    0.6943       100
          23     0.8842    0.8400    0.8615       100

    accuracy                   

In [42]:
# Обнаружено переобучение для прошлой итерации. 
task_3(192, 417, NUM_OF_CLASSES, 0.001, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  368
Ошибка в моменте переобучения:  0.5625549107789993
train
              precision    recall  f1-score   support

          39     0.8860    0.8860    0.8860       500
          58     0.8952    0.8880    0.8916       500
          23     0.9405    0.9480    0.9442       500

    accuracy                         0.9073      1500
   macro avg     0.9072    0.9073    0.9073      1500
weighted avg     0.9072    0.9073    0.9073      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.7143    0.8000    0.7547       100
          58     0.7356    0.6400    0.6845       100
          23     0.8416    0.8500    0.8458       100

    accuracy                   

In [43]:
# Создали новую модель FC(8), FC(3). 
seq = nn.Sequential(
            # полносвязный слой (входной - скрытый)
            nn.Linear(32*32*3, 8),
            # функция активации для нейронов скрытого слоя
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(8, NUM_OF_CLASSES),
        )
task_3(BATCH_SIZE, EPOCHS, NUM_OF_CLASSES, LEARNING_RATE, seq)

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  46
Ошибка в моменте переобучения:  0.6377203365166982
train
              precision    recall  f1-score   support

          39     0.9755    0.9540    0.9646       500
          58     0.9683    0.9780    0.9731       500
          23     0.9822    0.9940    0.9881       500

    accuracy                         0.9753      1500
   macro avg     0.9753    0.9753    0.9753      1500
weighted avg     0.9753    0.9753    0.9753      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6780    0.8000    0.7339       100
          58     0.6951    0.5700    0.6264       100
          23     0.8200    0.8200    0.8200       100

    accuracy                      

In [50]:
# Приведение к оптимальным гиперпараметрам.
seq = nn.Sequential(
            # полносвязный слой (входной - скрытый)
            nn.Linear(32*32*3, 8),
            # функция активации для нейронов скрытого слоя
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(8, NUM_OF_CLASSES),
        )
task_3(BATCH_SIZE, 52 * 5, NUM_OF_CLASSES, 0.001, seq)

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  240
Ошибка в моменте переобучения:  0.6285305817921957
train
              precision    recall  f1-score   support

          39     0.8305    0.8820    0.8555       500
          58     0.8588    0.7540    0.8030       500
          23     0.8717    0.9240    0.8971       500

    accuracy                         0.8533      1500
   macro avg     0.8537    0.8533    0.8518      1500
weighted avg     0.8537    0.8533    0.8518      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6842    0.7800    0.7290       100
          58     0.6988    0.5800    0.6339       100
          23     0.8350    0.8600    0.8473       100

    accuracy                     

In [51]:
# Создали новую модель FC(8), FC(8) FC(3). 
seq = nn.Sequential(
            nn.Linear(32*32*3, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(8, NUM_OF_CLASSES),
        )
task_3(BATCH_SIZE, EPOCHS, NUM_OF_CLASSES, LEARNING_RATE, seq)

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  99
Ошибка в моменте переобучения:  0.6560724973678589
train
              precision    recall  f1-score   support

          39     0.9793    0.9440    0.9613       500
          58     0.9392    0.9880    0.9630       500
          23     0.9919    0.9760    0.9839       500

    accuracy                         0.9693      1500
   macro avg     0.9701    0.9693    0.9694      1500
weighted avg     0.9701    0.9693    0.9694      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6726    0.7600    0.7136       100
          58     0.6264    0.5700    0.5969       100
          23   

In [56]:
# Приведение к оптимальным гиперпараметрам.
seq = nn.Sequential(
            nn.Linear(32*32*3, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(8, NUM_OF_CLASSES),
        )
task_3(BATCH_SIZE, 350, NUM_OF_CLASSES, 0.001, seq)

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  346
Ошибка в моменте переобучения:  0.6534637014071146
train
              precision    recall  f1-score   support

          39     0.7449    0.8700    0.8026       500
          58     0.7964    0.6180    0.6959       500
          23     0.8447    0.8920    0.8677       500

    accuracy                         0.7933      1500
   macro avg     0.7953    0.7933    0.7887      1500
weighted avg     0.7953    0.7933    0.7887      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6311    0.7700    0.6937       100
          58     0.6316    0.4800    0.5455       100
          23  

In [ ]:
task_3(BATCH_SIZE, EPOCHS, NUM_OF_CLASSES, LEARNING_RATE, default_sequention())